# Capao Urban Rate

This notebook aims to estimate the current urban growth rate for the place: Vale do Capão, Palmeiras, BA, Brasil. By using Google Earth satellite images and analyzing RGB pixel data, we can determine the growth rate in areas featuring houses, roads, construction sites, or where forests have been cleared for humans uses. For this case study, we have chosen COPERNICUS satellite images.

In [ ]:
import ee
import geemap as geemap
import pandas as pd

from utils.contants import PROJECT
from utils.features import get_coordinates

ee.Authenticate()
ee.Initialize(project=PROJECT)
geemap.ee_initialize()

## Feature Colletion 

In [ ]:
# Place location and geometry of interest
geo_point = ee.Geometry.Point([-41.501150593949305,-12.609558240448216])
geo_place = ee.Geometry.Polygon([
    [-41.51551178850023,-12.571410698153873],
    [-41.51392392076341,-12.575557425201367],
    [-41.51349476732103,-12.596499467766318],
    [-41.516777307708075,-12.605673846986134],
    [-41.513816148955634,-12.613023835365905],
    [-41.510018140990546,-12.61471995657374],
    [-41.5042460271905,-12.622069685275841],
    [-41.49767997952204,-12.633585929553394],
    [-41.495040685851386,-12.64577164489336],
    [-41.49165037365656,-12.65577940923046],
    [-41.48326410447515,-12.654326569148258],
    [-41.482418542332205,-12.629461766530518],
    [-41.484178071445974,-12.62054181330977],
    [-41.486409669346365,-12.610658309543654],
    [-41.4919457487531,-12.603664243777065],
    [-41.4923984315149,-12.595572883050577],
    [-41.48810689709107,-12.581960700605178],
    [-41.48832147381226,-12.568473457194342],
    [-41.512010743831794,-12.568347796317699],
    [-41.51551178850023,-12.571410698153873]
])

# Pre process urban features
df = pd.read_csv('./data/urban_features.csv')
raw_urban = df['.geo'].tolist()
urban = [get_coordinates(i) for i in raw_urban]

urban_features_list = [
  ee.Feature(ee.Geometry.Point(urban[i][0], urban[i][1]), {'class': 1 }) for i in range(len(urban))
]

# Pre process vegetation features
df = pd.read_csv('./data/vegetation_feature.csv')
raw_vegetation = df['.geo'].tolist()
vegetation = [get_coordinates(i) for i in raw_vegetation]

vegetation_features_list = [
  ee.Feature(ee.Geometry.Point(vegetation[i][0], vegetation[i][1]), {'class': 0 }) for i in range(len(vegetation))
]

# Feature collections
urban_features = ee.FeatureCollection(urban_features_list, 'urban')
vegetation_features = ee.FeatureCollection(vegetation_features_list, 'vegetation')

feature = urban_features.merge(vegetation_features)

# feature.getInfo()

# Dataset

In [ ]:
image = (
  ee.Image(
    ee.ImageCollection("COPERNICUS/S2_SR_HARMONIZED")
      .filterBounds(geo_point)
      .filterDate('2023-01-01', '2023-12-01')
      .sort('CLOUDY_PIXEL_PERCENTAGE')
      .first()
  )
)

bands = ['B2', 'B3', 'B4', 'B5', 'B6', 'B7'] 

label = 'class'

feature_collection = image.select(bands).sampleRegions(
  collection = feature,
  properties = [label],
  scale = 10,
  geometries = True
)
# feature_collection.getInfo()

## Randon Forest

In [ ]:
dataset = feature_collection.randomColumn()
training = dataset.filter('random <= 0.8')
validation = dataset.filter('random > 0.8')

cls = ee.Classifier.smileRandomForest(10).train(training, label, bands)

train_accuracy = cls.confusionMatrix()
validation_sample = validation.classify(cls)
validation_accuracy = validation_sample.errorMatrix(label, 'classification')

# display('Results', cls.explain().getInfo())
display('Confusion Matrix', train_accuracy.getInfo())
display('Training error matrix', train_accuracy)
display('Training overall accuracy', train_accuracy.accuracy())
display('Validation error matrix', validation_accuracy)
display('Validation accuracy', validation_accuracy.accuracy())



## Map view

In [49]:
classification = image.clip(geo_place).classify(cls)

map = geemap.Map(center=[-12.609558240448216,-41.501150593949305], zoom=15)

class_vis = {
    'min': 0,
    'max': 10,
    'palette': [
        '006400',
        'ffbb22',
        'ffff4c',
        'f096ff',
        'fa0000',
        'b4b4b4',
        'f0f0f0',
        '0064c8',
        '0096a0',
        '00cf75',
        'fae6a0',
    ],
}


# Adjust this value to increase or decrease the contrast
vis_params = {"bands": ['B2', 'B3', 'B4'],  min: 0, max: 2000, "gamma": 3}

map.add_layer(image, vis_params, 'Polygon')
map.add_layer(classification, class_vis, 'Classification')
# Display the map
map

Map(center=[-12.609558240448216, -41.501150593949305], controls=(WidgetControl(options=['position', 'transpare…